In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# 设置中文和负号正常显示
plt.rcParams['font.sans-serif']=['Microsoft YaHei','SimHei'] #可以配置多个的
plt.rcParams['axes.unicode_minus']=False #处理些特殊字符

# ====================== 1. 数据加载与预处理 ======================
df = pd.read_csv("../data/temperature.csv")
time = df['time'].values
u = df['volte'].values
y = df['temperature'].values

# 去除初始稳态段（假设前50点为稳态）
y_initial = np.mean(y[:50])
step_idx = np.argmax(u > u[0] * 1.1)  # 检测阶跃起始点
t_step = time[step_idx]
y_step = y[step_idx:]
t_step = time[step_idx:] - t_step  # 以阶跃时刻为时间起点

In [32]:
# ====================== 2. 系统辨识（带延迟的一阶模型） ======================
def identify_first_order_with_delay(t, y, y_initial, threshold=0.05):
    """辨识带延迟的一阶模型：G(s) = K e^(-Ls)/(Ts + 1)"""
    #计算比例系数 K
    delta_y = y[-1] - y_initial  # 稳态输出变化
    K = delta_y / (u.max() - 0)  # 增益（假设输入为阶跃信号）

    # 计算延迟时间 L（达到threshold*delta_y的时间）
    target = y_initial + threshold * delta_y
    L_mask = y > target
    if np.any(L_mask):
        L = t[L_mask][0]
    else:
        L = 0  # 无明显延迟

    # 计算时间常数 T（去除延迟后的63%响应时间）
    y_adj = y - y_initial
    y_adj = y_adj[L_mask] if L > 0 else y_adj
    t_adj = t[L_mask] if L > 0 else t
    if len(y_adj) == 0:
        T = 0
    else:
        target_T = 0.63 * delta_y
        T_mask = y_adj > target_T
        if np.any(T_mask):
            T = t_adj[T_mask][0] - L
        else:
            T = np.inf  # 未达到63%响应

    return K, L, T


K, L, T = identify_first_order_with_delay(t_step, y_step, y_initial, threshold=0.05)
print(f"辨识结果：K={K:.4f}, L={L:.2f}s, T={T:.2f}s")

辨识结果：K=9.8346, L=210.00s, T=2882.00s


In [34]:
#-----------------------------------------------------------------------------------------------------------------------
#2.验证辨识模型
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal


# 重新构建时间轴，仿真和原始数据对齐
t_sim = time.copy()

# 构建传递函数 G(s) = K / (T*s + 1)
num = [K]
den = [T, 1]

# 构建离散时间序列仿真信号
system = signal.TransferFunction(num, den)

# 用 step response 进行仿真（单位阶跃输入 1V），咱们手动乘 delta_u 来对齐实际阶跃
t_out, y_out_no_delay = signal.step(system, T=t_sim)

# 加入阶跃幅值 + 初始温度
delta_u = u.max()-0
y_out_no_delay = y_initial + delta_u * y_out_no_delay

# 人为加延迟：L_time 对应多少个样本点
dt = t_sim[1] - t_sim[0]  # 采样周期
delay_steps = int(L / dt)

# 构造延迟后的信号
y_out = np.concatenate([np.ones(delay_steps) * y_initial, y_out_no_delay[:-delay_steps]])
delta_y = y[-1] - y_initial
# 画对比图
# 改进的可视化
plt.figure(figsize=(12, 8))

# 绘制实际响应和模型响应
plt.plot(time, y, 'b-', linewidth=1.5, label='实际响应')
plt.plot(time[step_idx:], y_out, 'r--', linewidth=2, label=f'模型响应 (K={K:.2f}, T={T:.1f}s, L={L:.1f}s)')

# 标记阶跃起始点
plt.axvline(time[step_idx], color='g', linestyle=':', label=f'阶跃起始点 (t={time[step_idx]:.1f}s)')

# 添加误差分析
error = y[step_idx:] - y_out
rmse = np.sqrt(np.mean(error**2))
plt.text(0.05, 0.95, f'RMSE: {rmse:.2f}℃', transform=plt.gca().transAxes, 
         fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.8))

# 标注关键点
plt.scatter(time[step_idx], y[step_idx], color='green', s=80, zorder=5)
plt.scatter(time[step_idx] + L, y_initial + 0.05*delta_y, color='purple', s=80, zorder=5, label=f'延迟结束 (L={L:.1f}s)')
plt.scatter(time[step_idx] + L + T, y_initial + 0.632*delta_y, color='orange', s=80, zorder=5, label=f'63.2%响应点 (T={T:.1f}s)')

# 设置图形属性
plt.xlabel('时间 (s)', fontsize=12)
plt.ylabel('温度 (℃)', fontsize=12)
plt.title('加热炉系统辨识结果', fontsize=14)
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# 添加误差子图
plt.figure(figsize=(12, 3))
plt.plot(time[step_idx:], error, 'm-', label='模型误差')
plt.axhline(0, color='k', linestyle='--')
plt.fill_between(time[step_idx:], 0, error, where=(error>=0), facecolor='red', alpha=0.3)
plt.fill_between(time[step_idx:], 0, error, where=(error<0), facecolor='blue', alpha=0.3)
plt.xlabel('时间 (s)')
plt.ylabel('误差 (℃)')
plt.title(f'模型误差 (RMSE = {rmse:.2f}℃)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

In [36]:
!pip install pyswarm

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# 3.智能优化算法PSO优化PID参数

# import numpy as np
# import matplotlib.pyplot as plt
# from pyswarm import pso

# ====================== 1. 系统参数 ======================
# 传递函数参数（从辨识得到）
# K_process = K  # 系统增益
# T_process = T  # 时间常数
# L_process = L  # 延迟时间

# # ====================== 2. 系统仿真函数 ======================
# def simulate_pid_system(Kp, Ki, Kd, dt=1.0, total_time=10000):
#     """带延迟的闭环系统时间域仿真"""
#     n_steps = int(total_time/dt)
#     t = np.arange(0, total_time, dt)
    
#     # 初始化数组
#     y = np.zeros(n_steps)  # 过程输出
#     u = np.zeros(n_steps)  # 控制器输出
#     e = np.zeros(n_steps)  # 误差
#     integral = 0           # 积分项
#     prev_error = 0         # 上一次误差
    
#     # 计算延迟步数（带插值）
#     delay_steps = L_process / dt
#     delay_int = int(np.floor(delay_steps))
#     delay_frac = delay_steps - delay_int
    
#     # 历史输出缓存（用于延迟处理）
#     y_buffer = np.zeros(delay_int + 2) if delay_int > 0 else np.zeros(2)
    
#     # 被控对象模型参数
#     alpha = np.exp(-dt/T_process)
#     beta = K_process * (1 - alpha)
    
#     # 主循环
#     for k in range(1, n_steps):
#         # 设定值（阶跃变化）
#         setpoint = 35.0 if t[k] > 0 else 0
        
#         # 读取带延迟的输出（使用插值）
#         if delay_int > 0:
#             y_delayed = y_buffer[0] + delay_frac*(y_buffer[1] - y_buffer[0])
#         else:
#             y_delayed = y[k-1]  # 无延迟情况
        
#         # 计算误差
#         e[k] = setpoint - y_delayed
        
#         # PID计算
#         integral += e[k] * dt
#         derivative = (e[k] - prev_error) / dt if k > 1 else 0
        
#         # 计算控制量
#         u_unclamped = Kp*e[k] + Ki*integral + Kd*derivative
        
#         # 执行器饱和处理
#         u[k] = np.clip(u_unclamped, 0, 10)  # 电压限制在0-10V
        
#         # 抗饱和积分（back-calculation）
#         if Ki > 1e-6:
#             integral += (u[k] - u_unclamped) / Ki
        
#         # 更新过程模型
#         y[k] = alpha*y[k-1] + beta*u[k-1]
        
#         # 更新延迟缓冲区
#         if delay_int > 0:
#             y_buffer[:-1] = y_buffer[1:]
#             y_buffer[-1] = y[k]
        
#         prev_error = e[k]
    
#     return t, y, u

# # ====================== 3. 性能指标函数 ======================
# def performance_index(params):
#     Kp, Ki, Kd = params
#     t, y, u = simulate_pid_system(Kp, Ki, Kd)
    
#     # 找到设定值变化后的响应
#     setpoint = 35.0
#     mask = t > 0
#     y_set = y[mask]
#     t_set = t[mask]
    
#     # 计算性能指标
#     error = y_set - setpoint
#     u_diff = np.diff(u[mask])
    
#     # 主要指标
#     rise_time = t_set[np.argmax(y_set >= 0.9*setpoint)]  # 上升时间
#     settling_idx = np.where(np.abs(error) < 0.05*setpoint)[0]
#     settling_time = t_set[settling_idx[0]] if len(settling_idx) > 0 else t_set[-1]
#     overshoot = max(0, (np.max(y_set) - setpoint) / setpoint * 100)
    
#     # 误差积分指标
#     ITAE = np.sum(t_set * np.abs(error))
    
#     # 控制量变化惩罚
#     control_effort = np.sum(u_diff**2)
    
#     # 综合性能指标（权重可调整）
#     cost = ITAE/1e6 + 0.5*overshoot + 0.1*settling_time/100 + 0.01*control_effort/1e3
    
#     return cost

# # ====================== 4. 参数优化 ======================
# # 估算合理PID参数范围
# if L_process > 0:
#     Kp_max = min(5.0 * T_process/(K_process * L_process), 20.0)
# else:
#     Kp_max = 10.0

# Ki_max = 2.0 * Kp_max / T_process
# Kd_max = Kp_max * T_process

# lb = [0.01, 1e-6, 0]  # 下界
# ub = [Kp_max, Ki_max, Kd_max]  # 上界

# print("优化参数边界:")
# print(f"Kp: [{lb[0]:.4f}, {ub[0]:.4f}]")
# print(f"Ki: [{lb[1]:.6f}, {ub[1]:.6f}]")
# print(f"Kd: [{lb[2]:.4f}, {ub[2]:.4f}]")

# # PSO优化
# best_params, best_cost = pso(
#     performance_index, 
#     lb, 
#     ub, 
#     swarmsize=30, 
#     maxiter=50,
#     debug=True
# )

# Kp_opt, Ki_opt, Kd_opt = best_params
# print(f"\n优化结果: Kp={Kp_opt:.4f}, Ki={Ki_opt:.6f}, Kd={Kd_opt:.4f}")
# print(f"性能指标: {best_cost:.4f}")

# # ====================== 5. 结果验证与可视化 ======================
# # 最优参数仿真
# t, y_opt, u_opt = simulate_pid_system(Kp_opt, Ki_opt, Kd_opt)

# # 绘制结果
# plt.figure(figsize=(12, 8))

# # 温度响应
# plt.subplot(3, 1, 1)
# plt.plot(t, y_opt, 'b-', label='温度响应')
# plt.axhline(35, color='r', linestyle='--', label='设定值')
# plt.ylabel('温度 (℃)')
# plt.title(f'PID控制优化结果: Kp={Kp_opt:.3f}, Ki={Ki_opt:.5f}, Kd={Kd_opt:.3f}')
# plt.grid(True)
# plt.legend()

# # 控制电压
# plt.subplot(3, 1, 2)
# plt.plot(t, u_opt, 'g-', label='控制电压')
# plt.axhline(0, color='k', linestyle=':')
# plt.axhline(10, color='k', linestyle=':')
# plt.ylabel('电压 (V)')
# plt.ylim(-0.5, 10.5)
# plt.grid(True)

# # 误差
# plt.subplot(3, 1, 3)
# error = y_opt - 35.0
# mask = t > 0
# plt.plot(t[mask], error[mask], 'r-', label='误差')
# plt.axhline(0, color='k', linestyle='--')
# plt.xlabel('时间 (s)')
# plt.ylabel('误差 (℃)')
# plt.grid(True)

# plt.tight_layout()

# # 计算并显示性能指标
# settling_idx = np.where(np.abs(error[mask]) < 0.5)[0]
# settling_time = t[mask][settling_idx[0]] if len(settling_idx) > 0 else t[-1]
# overshoot = max(0, (np.max(y_opt[mask]) - 35) / 35 * 100)
# itae = np.sum(t[mask] * np.abs(error[mask]))

# plt.figtext(0.12, 0.92, f"超调: {overshoot:.1f}%", fontsize=10)
# plt.figtext(0.32, 0.92, f"调节时间: {settling_time:.0f}s", fontsize=10)
# plt.figtext(0.52, 0.92, f"稳态误差: {error[-1]:.2f}℃", fontsize=10)
# plt.figtext(0.72, 0.92, f"ITAE: {itae/1e6:.1f}×10⁶", fontsize=10)

# # 保存结果
# plt.savefig('pid_optimization_results.png', dpi=300)
# plt.show()

# 绘制优化过程（如果PSO返回历史数据）
# 注：标准pyswarm不返回历史，可考虑使用其他库或自定义PSO实现
import numpy as np
import matplotlib.pyplot as plt
from pyswarm import pso
import time as timer
from scipy.interpolate import interp1d

# ====================== 系统参数 ======================
# 示例参数（实际应用中应从外部获取）
K = 1.8    # 系统增益
T = 120.0  # 时间常数 (秒)
L = 45.0   # 延迟时间 (秒)

# 全局参数
K_process = K
T_process = T
L_process = L

# ====================== 系统仿真函数 ======================
def simulate_pid_system(Kp, Ki, Kd, dt=0.5, total_time=5000, setpoint=35.0):
    """带延迟的闭环系统时间域仿真（增强版）"""
    n_steps = int(total_time/dt)
    t = np.arange(0, total_time, dt)
    
    # 初始化数组
    y = np.zeros(n_steps)  # 过程输出
    u = np.zeros(n_steps)  # 控制器输出
    e = np.zeros(n_steps)  # 误差
    integral = 0           # 积分项
    prev_error = 0         # 上一次误差
    
    # 计算延迟步数（带插值）
    delay_steps = L_process / dt
    delay_int = int(np.floor(delay_steps))
    delay_frac = delay_steps - delay_int
    
    # 历史输出缓存（用于延迟处理）
    y_buffer = np.zeros(delay_int + 4) if delay_int > 0 else np.zeros(4)
    
    # 被控对象模型参数
    alpha = np.exp(-dt/T_process)
    beta = K_process * (1 - alpha)
    
    # 初始化设定值
    sp = np.zeros(n_steps)
    sp[:] = setpoint  # 全程设定值
    
    # 初始化性能指标
    overshoot = 0
    settling_time = total_time
    steady_error = 0
    itae = 0
    rise_time = total_time
    
    # 主循环
    for k in range(1, n_steps):
        # 读取带延迟的输出（使用插值）
        if delay_int > 0 and k > delay_int + 2:
            # 使用三次样条插值提高精度
            interp_points = y[k-delay_int-2:k-delay_int+2]
            interp_times = np.arange(-2, 2) * dt
            f = interp1d(interp_times, interp_points, kind='cubic', fill_value="extrapolate")
            y_delayed = f(-delay_frac * dt)
        elif delay_int > 0:
            # 线性插值
            y_delayed = y_buffer[0] + delay_frac*(y_buffer[1] - y_buffer[0])
        else:
            y_delayed = y[k-1]  # 无延迟情况
        
        # 计算误差
        e[k] = sp[k] - y_delayed
        
        # PID计算
        # 积分分离：大误差时禁用积分
        if abs(e[k]) > 5:  
            integral = integral
        else:
            integral += e[k] * dt
            
        derivative = (e[k] - prev_error) / dt if k > 1 else 0
        
        # 计算控制量
        u_unclamped = Kp*e[k] + Ki*integral + Kd*derivative
        
        # 执行器饱和处理
        u[k] = np.clip(u_unclamped, 0, 10)  # 电压限制在0-10V
        
        # 抗饱和积分（back-calculation）
        if Ki > 1e-6:
            integral += (u[k] - u_unclamped) * (1.0/Ki)  # 修正抗饱和计算
        
        # 更新过程模型
        y[k] = alpha*y[k-1] + beta*u[k-1]
        
        # 更新延迟缓冲区
        if delay_int > 0:
            y_buffer[:-1] = y_buffer[1:]
            y_buffer[-1] = y[k]
        
        prev_error = e[k]
    
    # 计算关键性能指标
    mask = t > 1  # 忽略初始过渡
    
    # 超调量
    y_max = np.max(y[mask])
    overshoot = max(0, (y_max - setpoint) / setpoint * 100) if setpoint > 0 else 0
    
    # 调节时间（进入±0.5℃误差带）
    error = np.abs(y - setpoint)
    in_band = error < 0.5
    settling_idx = None
    
    # 找到首次进入误差带且之后不再超出
    for i in range(len(t)):
        if t[i] > 50 and in_band[i]:  # 忽略前50秒
            # 检查之后是否都在误差带内
            if np.all(in_band[i:min(i+100, len(t))]):  # 检查后续100个点
                settling_idx = i
                break
    
    settling_time = t[settling_idx] if settling_idx is not None else total_time
    
    # 稳态误差（最后10%数据）
    steady_error = np.mean(y[-len(y)//10:] - setpoint)
    
    # ITAE指标
    itae = np.sum(t[mask] * np.abs(y[mask] - setpoint))
    
    # 上升时间（10%到90%）
    start_idx, end_idx = None, None
    for i in range(len(y)):
        if y[i] > 0.1 * setpoint and start_idx is None:
            start_idx = i
        if y[i] > 0.9 * setpoint and end_idx is None:
            end_idx = i
            break
    
    if start_idx is not None and end_idx is not None:
        rise_time = t[end_idx] - t[start_idx]
    else:
        rise_time = total_time
    
    return t, y, u, overshoot, settling_time, steady_error, itae, rise_time

# ====================== 性能指标函数 ======================
def performance_index(params):
    Kp, Ki, Kd = params
    try:
        _, _, _, overshoot, settling_time, steady_error, itae, rise_time = simulate_pid_system(
            Kp, Ki, Kd, total_time=3000  # 缩短仿真时间提高效率
        )
    except Exception as e:
        print(f"仿真失败: {e}")
        return float('inf')  # 返回极大值避免无效参数
    
    # 新的性能指标权重
    w_overshoot = 1.0
    w_settling = 3.0       # 显著增加调节时间权重
    w_steady = 2.0
    w_itae = 0.1           # 降低ITAE权重
    w_rise = 0.5           # 上升时间权重
    
    # 更严格的惩罚机制
    penalty = 0
    if overshoot > 2:      # 降低超调容忍度
        penalty += 50 * (overshoot - 2) ** 2  # 二次惩罚
    
    # 大幅降低调节时间容忍阈值
    if settling_time > 150:
        # 指数级惩罚
        penalty += 1000 * np.exp(0.05 * (settling_time - 150))
    
    if abs(steady_error) > 0.3:  # 提高稳态精度要求
        penalty += 100 * abs(steady_error)
    
    if rise_time > 100:  # 要求上升时间<100s
        penalty += 500 * (rise_time - 100)
    
    # 综合性能指标
    cost = (
        w_overshoot * overshoot +
        w_settling * (settling_time / 50) +  # 分母减小以放大影响
        w_steady * abs(steady_error) * 20 +  # 放大稳态误差影响
        w_itae * (itae / 1e6) +
        w_rise * (rise_time / 50) +
        penalty
    )
    
    # 对调节时间特别长的方案施加额外惩罚
    if settling_time > 500:
        cost *= 1.5
    
    return cost

# ====================== 参数优化 ======================
# 估算合理PID参数范围
if L_process > 0:
    Kp_max = min(15.0 * T_process/(K_process * L_process), 30.0)
else:
    Kp_max = 15.0

Ki_max = min(10.0 * Kp_max / T_process, 5.0)  # 提高积分上限
Kd_max = min(Kp_max * T_process, 15.0)

lb = [0.1, 1e-5, 0.1]  # 提高下界
ub = [Kp_max, Ki_max, Kd_max]  # 扩展上界

print("优化参数边界:")
print(f"Kp: [{lb[0]:.4f}, {ub[0]:.4f}]")
print(f"Ki: [{lb[1]:.6f}, {ub[1]:.6f}]")
print(f"Kd: [{lb[2]:.4f}, {ub[2]:.4f}]")

# 记录优化开始时间
start_time = timer.time()

# PSO优化（带详细输出）
print("\n开始PSO优化...")
best_params, best_cost = pso(
    performance_index, 
    lb, 
    ub, 
    swarmsize=50,      # 增加粒子数
    maxiter=100,       # 增加迭代次数
    phip=1.0,          # 增加认知参数
    phig=1.5,          # 增加社会参数
    omega=0.4,         # 降低惯性权重
    debug=True,
    minfunc=1e-6,
    minstep=1e-8
)

# 计算优化耗时
optim_time = timer.time() - start_time
print(f"\n优化完成! 耗时: {optim_time:.1f}秒")
Kp_opt, Ki_opt, Kd_opt = best_params
print(f"优化结果: Kp={Kp_opt:.4f}, Ki={Ki_opt:.6f}, Kd={Kd_opt:.4f}")
print(f"性能指标: {best_cost:.4f}")

# ====================== 详细结果分析与可视化 ======================
# 最优参数仿真（完整时间）
t, y_opt, u_opt, overshoot, settling_time, steady_error, itae, rise_time = simulate_pid_system(
    Kp_opt, Ki_opt, Kd_opt, total_time=10000
)

# 创建专业图表
plt.figure(figsize=(14, 10))
plt.suptitle(f'加热炉PID控制系统优化结果 (Kp={Kp_opt:.3f}, Ki={Ki_opt:.5f}, Kd={Kd_opt:.3f})', 
             fontsize=16, y=0.97)

# 温度响应 (主图)
ax1 = plt.subplot(3, 1, 1)
ax1.plot(t, y_opt, 'b-', linewidth=2, label='温度响应')
ax1.axhline(35, color='r', linestyle='--', label='设定值 (35℃)')

# 标记关键性能点
ax1.axvline(settling_time, color='m', linestyle='-.', label=f'调节时间 ({settling_time:.0f}s)')
ax1.scatter(t[np.argmax(y_opt)], np.max(y_opt), color='g', s=80, 
            label=f'峰值温度 ({np.max(y_opt):.2f}℃)')

# 标记上升时间
ax1.axvline(rise_time, color='c', linestyle=':', label=f'上升时间 ({rise_time:.0f}s)')

# 误差带 (±0.5℃)
ax1.fill_between(t, 34.5, 35.5, color='gray', alpha=0.2, label='±0.5℃误差带')

ax1.set_ylabel('温度 (℃)', fontsize=12)
ax1.set_ylim(30, max(40, np.max(y_opt)*1.05))
ax1.grid(True, linestyle='--', alpha=0.7)
ax1.legend(loc='lower right', fontsize=10)

# 控制电压
ax2 = plt.subplot(3, 1, 2, sharex=ax1)
ax2.plot(t, u_opt, 'g-', linewidth=1.5, label='控制电压')
ax2.axhline(0, color='k', linestyle=':')
ax2.axhline(10, color='k', linestyle=':', label='执行器限幅')
ax2.set_ylabel('电压 (V)', fontsize=12)
ax2.set_ylim(-0.5, 10.5)
ax2.grid(True, linestyle='--', alpha=0.5)
ax2.legend(loc='upper right', fontsize=10)

# 温度误差
ax3 = plt.subplot(3, 1, 3, sharex=ax1)
error = y_opt - 35.0
ax3.plot(t, error, 'r-', linewidth=1.2, label='温度误差')
ax3.axhline(0, color='k', linestyle='--')
ax3.fill_between(t, -0.5, 0.5, color='gray', alpha=0.2, label='±0.5℃误差带')
ax3.set_xlabel('时间 (s)', fontsize=12)
ax3.set_ylabel('误差 (℃)', fontsize=12)
ax3.grid(True, linestyle='--', alpha=0.5)
ax3.legend(loc='upper right', fontsize=10)

# 添加性能指标表格
performance_table = [
    ["性能指标", "数值", "要求"],
    ["超调量", f"{overshoot:.2f}%", "<2%"],
    ["调节时间", f"{settling_time:.0f}s", "<150s"],
    ["稳态误差", f"{steady_error:.3f}℃", "<0.3℃"],
    ["上升时间", f"{rise_time:.0f}s", "<100s"],
    ["ITAE指标", f"{itae/1e6:.1f}×10⁶", "最小化"],
    ["优化耗时", f"{optim_time:.1f}秒", "-"]
]

# 创建表格
ax_table = plt.gcf().add_axes([0.65, 0.45, 0.25, 0.2])
ax_table.axis('off')
table = ax_table.table(
    cellText=performance_table,
    colWidths=[0.3, 0.3, 0.3],
    cellLoc='center',
    loc='center'
)

# 设置表格样式
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

# 突出显示满足要求的指标
for i in range(1, len(performance_table)):
    if i == 1 and overshoot <= 2:
        table[(i, 1)].set_facecolor('#90EE90')  # 浅绿
    elif i == 2 and settling_time <= 150:
        table[(i, 1)].set_facecolor('#90EE90')
    elif i == 3 and abs(steady_error) <= 0.3:
        table[(i, 1)].set_facecolor('#90EE90')
    elif i == 4 and rise_time <= 100:
        table[(i, 1)].set_facecolor('#90EE90')

plt.tight_layout(rect=[0, 0, 1, 0.95])

# 保存结果
plt.savefig('pid_optimization_results.png', dpi=300, bbox_inches='tight')
plt.show()

# ====================== 鲁棒性测试 ======================
print("\n进行鲁棒性测试...")
test_cases = [
    ("标称工况", K, T, L),
    ("增益+20%", K * 1.2, T, L),
    ("增益-20%", K * 0.8, T, L),
    ("时间常数+20%", K, T * 1.2, L),
    ("时间常数-20%", K, T * 0.8, L),
    ("延迟+20%", K, T, L * 1.2),
    ("延迟-20%", K, T, L * 0.8)
]

plt.figure(figsize=(12, 8))
for i, (name, K_test, T_test, L_test) in enumerate(test_cases):
    # 临时修改系统参数
    global K_process, T_process, L_process
    K_process, T_process, L_process = K_test, T_test, L_test
    
    # 仿真系统响应
    t, y, _, overshoot, settling_time, steady_error, _, rise_time = simulate_pid_system(
        Kp_opt, Ki_opt, Kd_opt, total_time=5000
    )
    
    # 绘制响应曲线
    plt.plot(t, y, label=f"{name} (超调:{overshoot:.1f}%, 调节:{settling_time:.0f}s, 上升:{rise_time:.0f}s)")
    
    # 恢复原始参数
    K_process, T_process, L_process = K, T, L

plt.axhline(35, color='r', linestyle='--', label='设定值')
plt.xlabel('时间 (s)')
plt.ylabel('温度 (℃)')
plt.title('PID控制器鲁棒性测试')
plt.legend(fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.savefig('pid_robustness_test.png', dpi=300)
plt.show()

# ====================== 参数敏感性分析 ======================
print("\n进行参数敏感性分析...")

def sensitivity_analysis(param_range, param_name, index=1):
    """分析参数对调节时间的影响"""
    results = []
    for value in param_range:
        if param_name == 'Kp':
            params = [value, Ki_opt, Kd_opt]
        elif param_name == 'Ki':
            params = [Kp_opt, value, Kd_opt]
        else:
            params = [Kp_opt, Ki_opt, value]
            
        _, _, _, _, settling_time, _, _, _ = simulate_pid_system(*params)
        results.append(settling_time)
    
    plt.figure(figsize=(10, 6))
    plt.plot(param_range, results, 'o-', linewidth=2, markersize=8)
    plt.xlabel(param_name, fontsize=12)
    plt.ylabel('调节时间 (s)', fontsize=12)
    plt.title(f'调节时间对{param_name}的敏感性', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # 标记最优值
    if param_name == 'Kp':
        opt_value = Kp_opt
    elif param_name == 'Ki':
        opt_value = Ki_opt
    else:
        opt_value = Kd_opt
        
    min_time = min(results)
    min_idx = np.argmin(results)
    plt.axvline(opt_value, color='r', linestyle='--', label=f'优化值={opt_value:.4f}')
    plt.axvline(param_range[min_idx], color='g', linestyle=':', label=f'最优值={param_range[min_idx]:.4f}')
    
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'sensitivity_{param_name}.png', dpi=300)
    plt.show()
    
    return param_range[min_idx], min_time

# 分析Kp对调节时间的影响
kp_range = np.linspace(max(0.5*Kp_opt, lb[0]), min(1.5*Kp_opt, ub[0]), 20)
best_kp, best_settling_kp = sensitivity_analysis(kp_range, 'Kp')

# 分析Ki对调节时间的影响
ki_range = np.linspace(max(0.5*Ki_opt, lb[1]), min(1.5*Ki_opt, ub[1]), 20)
best_ki, best_settling_ki = sensitivity_analysis(ki_range, 'Ki')

# 分析Kd对调节时间的影响
kd_range = np.linspace(max(0.5*Kd_opt, lb[2]), min(1.5*Kd_opt, ub[2]), 20)
best_kd, best_settling_kd = sensitivity_analysis(kd_range, 'Kd')

print("\n参数敏感性总结:")
print(f"Kp优化值: {Kp_opt:.4f}, 最优值: {best_kp:.4f}, 调节时间: {best_settling_kp:.1f}s")
print(f"Ki优化值: {Ki_opt:.6f}, 最优值: {best_ki:.6f}, 调节时间: {best_settling_ki:.1f}s")
print(f"Kd优化值: {Kd_opt:.4f}, 最优值: {best_kd:.4f}, 调节时间: {best_settling_kd:.1f}s")

优化参数边界:
Kp: [0.1000, 22.2222]
Ki: [0.000010, 1.851852]
Kd: [0.1000, 15.0000]

开始PSO优化...
No constraints given.


In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
#4.性能指标计算模板
def compute_performance_metrics(t_out, y_out, setpoint, tolerance=0.02):
    # 上升时间（5% 到 95%）
    y_5 = setpoint * 0.05
    y_95 = setpoint * 0.95

    try:
        t_rise_start = t_out[np.where(y_out >= y_5)[0][0]]
        t_rise_end = t_out[np.where(y_out >= y_95)[0][0]]
        rise_time = t_rise_end - t_rise_start
    except:
        rise_time = np.nan  # 如果没达到 95%，就 NaN

    # 超调量
    max_temp = np.max(y_out)
    overshoot = ((max_temp - setpoint) / setpoint) * 100

    # 稳态误差
    y_steady_state = np.mean(y_out[-100:])  # 最后100个样本点
    steady_state_error = abs(setpoint - y_steady_state)

    # 调整时间
    lower_bound = setpoint * (1 - tolerance)
    upper_bound = setpoint * (1 + tolerance)

    settling_time = np.nan  # 默认NaN
    for i in range(len(y_out)):
        if np.all((y_out[i:] >= lower_bound) & (y_out[i:] <= upper_bound)):
            settling_time = t_out[i]
            break

    # 返回结果
    metrics = {
        "Rise Time (s)": rise_time,
        "Overshoot (%)": overshoot,
        "Steady-State Error (°C)": steady_state_error,
        "Settling Time (s)": settling_time
    }

    return metrics



metrics = compute_performance_metrics(t_out, y_out, setpoint)

# 打印性能指标
for k, v in metrics.items():
    print(f"{k}: {v:.2f}")